# MNIST Dataset

## Init Spark 

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("MNIST") \
    .getOrCreate()

## Credits
- [abulbasar](https://github.com/abulbasar/pyspark-examples)
- [MNIST](http://yann.lecun.com/exdb/mnist/)

## Dataset

In [ ]:
!wget http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz

In [ ]:
!gzip -d *.gz

In [ ]:
!mkdir mnist-data

In [ ]:
!mv *-ubyte mnist-data

## Convert Dataset

- https://pjreddie.com/projects/mnist-in-csv/

In [ ]:
def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()

In [ ]:
convert("./mnist-data/train-images-idx3-ubyte", "./mnist-data/train-labels-idx1-ubyte", "./mnist-data/mnist_train.csv", 60000)
convert("./mnist-data/t10k-images-idx3-ubyte", "./mnist-data/t10k-labels-idx1-ubyte", "./mnist-data/mnist_test.csv", 10000)

## Path Setup

In [ ]:
import os
path = os.getcwd()
print(path)

## Read Data as DataFrames

In [ ]:
df_training = (spark
               .read
               .options(header = False, inferSchema = True)
               .csv(f"file://{path}/mnist-data/mnist_train.csv"))

In [ ]:
df_training.count()

## Transforming Features to Vectors

In [ ]:
print("No of columns: ", len(df_training.columns), df_training.columns)

In [ ]:
feature_culumns = ["_c" + str(i+1) for i in range(784)]
print(feature_culumns)

In [ ]:
from pyspark.ml.feature import VectorAssembler
vectorizer = VectorAssembler(inputCols=feature_culumns, outputCol="features")
training = (vectorizer
            .transform(df_training)
            .select("_c0", "features")
            .toDF("label", "features")
            .repartition(15)
            .cache())
training.show()

## Visualize the Data

In [ ]:
a = training.first().features.toArray()
type(a)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(a.reshape(28, 28), cmap="Greys")

In [ ]:
images = training.sample(False, 0.01, 1).take(25)
fig, _ = plt.subplots(5, 5, figsize = (10, 10))
for i, ax in enumerate(fig.axes):
    r = images[i]
    label = r.label
    features = r.features
    ax.imshow(features.toArray().reshape(28, 28), cmap = "Greys")
    ax.set_title("True: " + str(label))

plt.tight_layout()

## Distribution of Digits

In [ ]:
counts_df = training.groupBy("label").count().orderBy("label")

In [ ]:
counts_df.show(10)

In [ ]:
counts = counts_df.rdd.map(lambda r: {"label": r['label'], "count": r['count']}).collect()

In [ ]:
counts

In [ ]:
import pandas as pd
pd.DataFrame(counts).set_index("label").sort_index().plot.bar()

## Loading Test Data

In [ ]:
df_testing = (spark
              .read
              .options(header = False, inferSchema = True)
              .csv(f"file://{path}/mnist-data/mnist_test.csv"))

testing = (vectorizer
           .transform(df_testing)
           .select("_c0", "features")
           .toDF("label", "features")
           .cache())

In [ ]:
testing.show(2)

## Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol="features", 
                        labelCol="label", 
                        regParam=0.1, 
                        elasticNetParam=0.1, 
                        maxIter=10000)

In [ ]:
lr_model = lr.fit(training)

## Predict 

In [ ]:
from pyspark.sql.functions import expr

In [ ]:
test_pred = lr_model.transform(testing).withColumn("matched", expr("label == prediction"))
test_pred.show()

## Evaluate 

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

In [ ]:
evaluator.evaluate(test_pred)

In [ ]:
from pyspark.sql.functions import avg
test_pred \
 .withColumn("matched", expr("cast(matched as int)")) \
 .groupby("label") \
 .agg(avg("matched")) \
 .orderBy("label") \
 .show()

In [ ]:
wrong_df = test_pred.filter("matched = false")

In [ ]:
wrong_df.count()

In [ ]:
images = wrong_df.take(36)

In [ ]:
len(images)

In [ ]:
fig, _ = plt.subplots(6, 6, figsize = (20, 20))
for i, ax in enumerate(fig.axes):
    r = images[i]
    label = r.label
    prediction = int(r.prediction)
    features = r.features
    ax.imshow(features.toArray().reshape(28, 28), cmap = "Greys")
    ax.set_title(f"True: {str(label)} / Pred: {str(prediction)}")

plt.show()

## Stop Spark 

In [ ]:
spark.stop()